### Step 1: Install libraries 

In [ ]:
pip install shap xgboost boto3 mlflow

### Step 2: Import everything

In [ ]:
import xgboost
import shap
import mlflow
from sklearn.model_selection import train_test_split

### Step 3: Set experiment

In [ ]:
mlflow.set_experiment('demo-new')  

### Step 4: Get data

In [ ]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Step 5: Train model

In [ ]:
# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [ ]:
# view data sample
X_test.head(5)

### Step 6: Make a run

In [ ]:
with mlflow.start_run() as run:
   model_info = mlflow.sklearn.log_model(model, "model")
   result = mlflow.evaluate(
       model_info.model_uri,
       eval_data,
       targets="target",
       model_type="classifier",
       dataset_name="adult",
       evaluators=["default"],
   )

### Step 7: Make changes and re-run

In [ ]:
# re-train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.53, random_state=52)
model = xgboost.XGBClassifier().fit(X_train, y_train)

with mlflow.start_run() as run:
   model_info = mlflow.sklearn.log_model(model, "model")
   result = mlflow.evaluate(
       model_info.model_uri,
       eval_data,
       targets="target",
       model_type="classifier",
       dataset_name="adult",
       evaluators=["default"],
   )

### Step 8: Run a service

In [ ]:
# test request
{
  "data": {
    "names": [
       "Age", "Workclass", "Education-Num", "Marital Status", "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss", "Hours per week", "Country"
    ],
    "tensor": {
      "shape": [
        1,
        12
      ],
      "values": [
        27.0, 4, 10.0, 0, 1, 0, 4, 0, 0.0, 0.0, 38.0, 39
      ]
    }
  }
}